# web crawling
- 1. requests : json : 동적페이지(url의 변경 없이 데이터를 수정 및 추가)
- 2. requests : html : 정적페이지(url 변경으로 데이터를 수정 및 추가)
- 3. selenium : web browser : 1, 2번 방법을 사용하지 못할 때 사용
    - requests를 통해 요청시 서버에 요청이 많아져(파이썬으로) 차단 가능성 있음.
    - selenium으로 하면 웬만해선 다 크롤링이 가능하나 단점이 속도가 느리다.
    - 1->2->3번순으로 해보자. (1이 안되면 2번, 2가 안되면 3번...)
---
- 웹크롤링을 하고자 할 때, 혹 그 사이트에 모바일 페이지가 있는지 확인해보자. 모바일 페이지가 pc페이지보다 상대적으로 간단하기에 편하다

# 크롤링 연습

1. 네이버 주식 데이터 크롤링
    - 코스피 데이터 수집
    - 코스닥 데이터 수집
    - USD 환율 데이터 수집
    - 그래프, 상관계수 확인

- 크롤링 절차
    1. 웹 서비스 분석 : URL을 확인
    2. requests, response : json 데이터(문자열)를 얻기
    3. json 데이터(문자열) -> dict(parsing) -> 데이터 프레임

In [18]:
import requests
import pandas as pd

In [ ]:
# https://m.stock.naver.com/sise/siseIndex.nhn?code=KOSPI
# 개발자도구(F12) -> Newtork -> XHR
# https://m.stock.naver.com/api/json/sise/dailySiseIndexListJson.nhn?code=KOSPI&pageSize=20&page=6

In [5]:
# 1. 웹 서비스 분석 : URL 찾기
code, page_size, page = "KOSPI", 20, 1

url = 'https://m.stock.naver.com/api/json/sise/dailySiseIndexListJson.nhn?code={}&pageSize={}&page={}'\
.format(code, page_size, page)

print(url)

https://m.stock.naver.com/api/json/sise/dailySiseIndexListJson.nhn?code=KOSPI&pageSize=20&page=1


In [6]:
# 2. requests, response : JSON(str)
response = requests.get(url)
response

<Response [200]>

In [8]:
type(response.text) # response(json)의 타입

str

In [19]:
# 3. JSON(str) -> JSON(dict) -> DF
# response.text
# 텍스트 데이터를 딕셔너리로 파싱해주는 함수가 있다.
datas = response.json()['result']['siseList']
kospi_df = pd.DataFrame(datas)
kospi_df.tail(3)

,cd,dt,ncv,cv,cr,ov,hv,lv
17,KOSPI,20210512,3161.66,-47.77,-1.49,3209.36,3212.83,3138.04
18,KOSPI,20210511,3209.43,-39.87,-1.23,3239.92,3239.92,3192.25
19,KOSPI,20210510,3249.30,52.10,1.63,3201.46,3255.90,3201.46


In [22]:
# 함수로 만들기
def get_stock_data(code, page_size=20, page=1) :
    url = 'https://m.stock.naver.com/api/json/sise/dailySiseIndexListJson.nhn?code={}&pageSize={}&page={}'\
.format(code, page_size, page)
    response = requests.get(url)
    datas = response.json()['result']['siseList']
    return pd.DataFrame(datas)

In [28]:
kosdaq_df = get_stock_data('KOSDAQ',100)
kosdaq_df.tail(3)

,cd,dt,ncv,cv,cr,ov,hv,lv
97,KOSDAQ,20210114,980.29,1.16,0.12,979.10,980.29,973.63
98,KOSDAQ,20210113,979.13,5.41,0.56,974.14,982.66,972.10
99,KOSDAQ,20210112,973.72,-2.91,-0.30,977.14,982.46,956.72


In [24]:
kospi_df = get_stock_data('KOSPI', 100)
kospi_df.tail(3)

,cd,dt,ncv,cv,cr,ov,hv,lv
97,KOSPI,20210114,3149.93,1.64,0.05,3148.65,3159.03,3128.72
98,KOSPI,20210113,3148.29,22.34,0.71,3128.26,3164.37,3109.63
99,KOSPI,20210112,3125.95,-22.50,-0.71,3145.87,3154.79,3047.56


In [ ]:
### USD 수집

In [31]:
# 1. 웹 페이지 분석 : URL 확인
code, page_size, page = 'FX_USDKRW', 20, 1
url = 'https://api.stock.naver.com/marketindex/exchange/{}/prices?page={}&pageSize={}'.format(code, page, page_size)
print(url)

https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page=1&pageSize=20


In [41]:
# 2. requests, response : JSON(str)
# 쿠키 데이터 추가해서 요청
params = {
    "cookie" : "NNB=K54L4BL7ARCV6; ASID=3d48494000000174259caa8700000055; NRTK=ag#all_gr#0_ma#-2_si#-2_en#-2_sp#-2; _fbp=fb.1.1615788718433.606469894; nx_ssl=2; _ga=GA1.2.910698097.1600608134; _ga_7VKFYR6RV1=GS1.1.1622736178.12.0.1622736178.60; NDARK=N; _gid=GA1.2.517172928.1623026828; MM_NEW=1; NFS=2; page_uid=h71xTdprvmZssScU5hossssssCV-019592; BMR=s=1623028403907&r=https%3A%2F%2Fm.stock.naver.com%2Findex.nhn&r2=https%3A%2F%2Fm.stock.naver.com%2Fsise%2FsiseIndex.nhn%3Fcode%3DKOSDAQ"
}



response = requests.get(url, headers=params)
response

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# 3. JSON(str) -> DF

In [ ]:
NNB=K54L4BL7ARCV6; ASID=3d48494000000174259caa8700000055; NRTK=ag#all_gr#0_ma#-2_si#-2_en#-2_sp#-2; _fbp=fb.1.1615788718433.606469894; nx_ssl=2; _ga=GA1.2.910698097.1600608134; _ga_7VKFYR6RV1=GS1.1.1622736178.12.0.1622736178.60; NDARK=N; _gid=GA1.2.517172928.1623026828; MM_NEW=1; NFS=2; page_uid=h71xTdprvmZssScU5hossssssCV-019592; BMR=s=1623028403907&r=https%3A%2F%2Fm.stock.naver.com%2Findex.nhn&r2=https%3A%2F%2Fm.stock.naver.com%2Fsise%2FsiseIndex.nhn%3Fcode%3DKOSDAQ